In [ ]:
!pip install langchain
!pip install langchain_chroma
!pip install sentence_transformers
!pip show langchain
!pip install aiofiles
!pip install datasets
!pip install accelerate

In [ ]:
!pip install datasets

In [ ]:
!pip install aiofiles

In [ ]:
import pandas as pd
df=pd.read_csv("test.csv",encoding='latin1')
df=df.head(200)
df.to_csv("test.csv")

In [ ]:
df

In [ ]:
import csv
from langchain.docstore.document import Document

def read_csv_to_langchain_documents(csv_file_path):
  """
  Reads a CSV file and converts each row into a LangChain Document object.

  Args:
      csv_file_path (str): The path to the CSV file.

  Returns:
      list: A list of LangChain Document objects.
  """

  documents = []
  try:
    with open(csv_file_path, 'r', newline='') as csvfile:
      reader = csv.DictReader(csvfile)

      for row in reader:
        # Construct page content (adjust column names and processing as needed)
        content = row["text"]

                # Create metadata using all other columns
        metadata = {key: value for key, value in row.items() if key != "text"}

        documents.append(Document(page_content=content, metadata=metadata))

  except FileNotFoundError:
    print(f"Error: CSV file '{csv_file_path}' not found.")
  except Exception as e:
    print(f"An error occurred while processing the CSV file: {str(e)}")

  return documents



In [ ]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import CharacterTextSplitter



documents = read_csv_to_langchain_documents("test.csv")


embedding_function = SentenceTransformerEmbeddings(model_name="thenlper/gte-small")

db = Chroma.from_documents(documents, embedding_function)



In [ ]:

query = "yo"
docs = db.similarity_search(query)
print("docs")
filter_metadata = {"sentiment": "negative"}
filtered_documents = db.filter(where=filter_metadata)

print("Page Content:")
print(filtered_documents[0].page_content)
print("Meta Data:")
print(filtered_documents[0].metadata)

In [ ]:
from typing import Optional

from langchain.chains.query_constructor.ir import (
    Comparator,
    Comparison,
    Operation,
    Operator,
    StructuredQuery,
)
from langchain.retrievers.self_query.chroma import ChromaTranslator
from langchain.retrievers.self_query.elasticsearch import ElasticsearchTranslator
from langchain_core.pydantic_v1 import BaseModel







class Search(BaseModel):
    query: str
    sentiment:str
    country: str

search_query = Search(query="RAG", sentiment="positive", country="Afghanistan")


def construct_comparisons(query: Search):
    comparisons = []
    if query.sentiment is not None:
        comparisons.append(
            Comparison(
                comparator=Comparator.GT,
                attribute="sentiment",
                value=query.sentiment,
            )
        )
    if query.country is not None:
        comparisons.append(
            Comparison(
                comparator=Comparator.EQ,
                attribute="country",
                value=query.country,
            )
        )
    return comparisons


comparisons = construct_comparisons(search_query)
_filter = Operation(operator=Operator.AND, arguments=comparisons)

ChromaTranslator().visit_operation(_filter)




In [ ]:
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)


In [20]:
document_text = "Replace me by any text you'd like."



def perform_rag_analysis(document_text):
    # Encode the document text
    encoded_input = tokenizer(document_text, return_tensors='pt')

    # Generate text continuation
    with torch.no_grad():
        outputs = model(**encoded_input)

    # Extract relevant features from the generated text (e.g., sentiment, keyword presence)
    # For example, you could use a sentiment analysis model or keyword extraction techniques here

    # Perform RAG classification based on the extracted features
    # Here, we use a simple placeholder logic
    rag_status = "Green"  # Placeholder: replace with actual RAG classification logic

    return rag_status

# Get RAG status for the document
document_rag_status = perform_rag_analysis(document_text)
print("RAG status:", document_rag_status)

RAG status: Green


In [ ]:
output_dir = "/microsoftLLM"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

saved_model_dir = "./LLM"
model = AutoModelForCausalLM.from_pretrained(saved_model_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_model_dir)